Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [2]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [9]:
def query_string(start_lat, start_long, dest_lat, dest_long,
                 transit = True, bus = True, train = True, bike = True, walk_dist = 5000, 
                 optimizeQuick = True,
                 walk_reluc = 2, visual = False, max_transfers = 3):
    base = 'http://vta2.amigocloud.com'
    api = '/api/otp/plan?'
    date = 'date=2016-07-13'
    ntime = '&time=09%3A0'
    #time = '&time=' + time
    
    if bike:
        mode = '&mode=BICYCLE%2C'
    else:
        mode = '&mode=WALK%2C'
    
    if transit:
        mode = mode + 'TRANSIT'
    else:
        if bus:
            mode = mode + 'BUSISH%2C'
        if train:
            mode = mode + 'TRAINISH'
    #mode = '&mode=WALK%2CBUSISH%2CTRAINISH'
    
    start = '&fromPlace=' + str(start_lat) + '%2C' + str(start_long)
    #start = '&fromPlace=37.389268%2C-121.879746'
    dest = '&toPlace=' + str(dest_lat) + '%2C' + str(dest_long)
    #dest = '&toPlace=37.4031595%2C-121.9865994'
    num_iter = '&numItineraries=3'
    
    walk = '&maxWalkDistance=' + str(walk_dist) + '&'
    
    bike = 'bikeSpeed=10'
    
    
    walk_reluctance = '&walkReluctance=' + str(walk_reluc)
    max_tran = '&maxTransfers=' + str(max_transfers)
    
    if optimizeQuick:
        triangle = '&optimize=QUICK'
    else:
        triangle = '&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE'
    
    arriveby = '&arriveBy=false'
    
    #Visualize doesn't work right at the moment
    if visual:
        api = '/planner?'
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby
    else:
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby

    return query

In [10]:
def print_transit(query_json):
    for a in query_json['plan']['itineraries']:
        print('start: '+ tclean(a['startTime']))
        print('end: '+ tclean(a['endTime']))
        print('duration: '+ str(a['duration']/60) + ' minutes')
        print('waitingTime: '+ str(a['waitingTime']/60) + ' minutes')
        print('transitTime: '+ str(a['transitTime']/60) + ' minutes')
        print('walkDistance: '+ str(a['walkDistance']/1000*.621371) + ' miles')
        print('transfers: '+ str(a['transfers']))
        for each in a['legs']:
            print('   StartTime: '+ tclean(each['startTime']) + " mode: " + each['mode'])
            print("   headsign: " + str(each.get('headsign')))
            print('   endTime:   ' + tclean(each['endTime']) + " duration: " + str(each['duration']/60))    

In [11]:
import os
os.getcwd()

'/Users/Vivek/Github/OTP_Tester'

In [12]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')

df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [13]:
def tclean(dto):
    return str(datetime.fromtimestamp(dto/1000))

In [14]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose
2,37.400787,-121.868619,1684 Garvey Pl,37.400281,-121.939691,3331 N. 1st St VTA
3,37.400281,-121.939691,3331 N. 1st St VTA,37.333539,-121.885089,"116 Paseo De San Antonio Walk, San Jose, CA 95..."


In [15]:
#visuals = []
#visuals.append(query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row], walk_dist = 10000,visual = True))
#df['visual'] = visuals

In [16]:
def input_params(df, max_walk, walk_reluc_factor, transit = True, bus_v = True, train_v = True, bike_v = True):
    row = 0
    times=[]
    results = []
    queries = []
    trips = []
    lowest = []
    
    for each in df.iterrows():
        query = query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], 
                             dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row],
                             walk_dist = max_walk, walk_reluc = walk_reluc_factor, 
                             transit = transit, bus = bus_v, train = train_v, bike = bike_v)
        queries.append(query)
        r = requests.get(query)
        query_json = json.loads(r.text)
        results.append(str(query_json))
        one_set_of_times= query_json['debugOutput']['pathTimes']
        times.append(one_set_of_times)
        row = row + 1
    df['times'] = times
    
    
    #Calculate Lowest trip length
    for each in df['times']:
        trips.append(len(each))
        if len(each) != 0:
            lowest.append(min(each))
        else:
            lowest.append(999)
    df['trips']=trips
    df['lowest']=lowest
    
    df['queries'] = queries
    df['results'] = results
    
    
    return df

In [60]:
series=input_params(df, max_walk = 5000, walk_reluc_factor = 10, transit = False, train_v = True, bike_v = False, bus_v = True)
query = series['queries'][1]
r = requests.get(query)
query_json = json.loads(r.text)
print_transit(query_json)

start: 2016-07-13 09:14:39
end: 2016-07-13 09:47:48
duration: 33.15 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 25.5 minutes
walkDistance: 0.3550073748388413 miles
transfers: 0
   StartTime: 2016-07-13 09:14:39 mode: WALK
   headsign: None
   endTime:   2016-07-13 09:18:59 duration: 4.333333333333333
   StartTime: 2016-07-13 09:19:00 mode: TRAM
   headsign: SANTA TERESA
   endTime:   2016-07-13 09:44:30 duration: 25.5
   StartTime: 2016-07-13 09:44:31 mode: WALK
   headsign: None
   endTime:   2016-07-13 09:47:48 duration: 3.283333333333333
start: 2016-07-13 09:24:39
end: 2016-07-13 09:57:48
duration: 33.15 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 25.5 minutes
walkDistance: 0.3550073748388413 miles
transfers: 0
   StartTime: 2016-07-13 09:24:39 mode: WALK
   headsign: None
   endTime:   2016-07-13 09:28:59 duration: 4.333333333333333
   StartTime: 2016-07-13 09:29:00 mode: TRAM
   headsign: WINCHESTER
   endTime:   2016-07-13 09:54:30 duration: 25

In [53]:
query

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-13&time=09%3A0&mode=WALK%2CTRANSIT&fromPlace=37.400281%2C-121.939691&toPlace=37.3335387%2C-121.8850891&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=3&optimize=QUICK&arriveBy=false'

In [67]:
series=input_params(df, max_walk = 1207, walk_reluc_factor = 15, transit = False, train_v = True, bike_v = False, bus_v = False)
query = series['queries'][1]
r = requests.get(query)
query_json = json.loads(r.text)
print_transit(query_json)

KeyError: 'plan'

In [3]:
>>> payload = {'key1': 'value1', 'key2': 'value2'}
>>> r = requests.get('http://httpbin.org/get', params=payload)

In [5]:
payload = {'date': '2016-07-13', 'mode': ['WALK','BUSISH','TRAINISH']}
r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
r.text

'{"requestParameters":{"mode":"WALK","date":"2016-07-13"},"error":{"id":400,"msg":"Trip is not possible.  You might be trying to plan a trip outside the map data boundary.","message":"OUTSIDE_BOUNDS","missing":["from","to"],"noPath":true},"debugOutput":{"precalculationTime":0,"pathCalculationTime":0,"pathTimes":[],"renderingTime":0,"totalTime":0,"timedOut":false}}'

In [54]:
payload.update({'fromPlace': str(df['start_lat'][0]) + ',' + str(df['start_long'][0])})

In [57]:
payload.update({'toPlace':str(df['dest_lat'][0]) + ',' + str(df['dest_long'][0])})

In [59]:
str(['WALK','BUSISH','TRAINISH'])

"['WALK', 'BUSISH', 'TRAINISH']"

In [56]:
r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
r.text

'{"requestParameters":{"mode":"WALK","date":"2016-07-13","fromPlace":"37.394535,-121.868908","toPlace":"37.400281,-121.939691"},"plan":{"date":1468450661000,"from":{"name":"corner of Camino del Rey and North Capitol Avenue","lon":-121.868908,"lat":37.394535,"orig":"","vertexType":"NORMAL"},"to":{"name":"parking aisle","lon":-121.93975519745304,"lat":37.40064728290023,"orig":"","vertexType":"NORMAL"},"itineraries":[{"duration":6429,"startTime":1468450661000,"endTime":1468457090000,"walkTime":6429,"transitTime":0,"waitingTime":0,"walkDistance":8374.270706757794,"walkLimitExceeded":true,"elevationLost":0.0,"elevationGained":0.0,"transfers":0,"legs":[{"startTime":1468450661000,"endTime":1468457090000,"departureDelay":0,"arrivalDelay":0,"realTime":false,"distance":8373.988,"pathway":false,"mode":"WALK","route":"","agencyTimeZoneOffset":-25200000,"interlineWithPreviousLeg":false,"from":{"name":"corner of Camino del Rey and North Capitol Avenue","lon":-121.868908,"lat":37.394535,"departure":1

In [58]:
r.json()

{'debugOutput': {'pathCalculationTime': 27,
  'pathTimes': [27],
  'precalculationTime': 0,
  'renderingTime': 1,
  'timedOut': False,
  'totalTime': 28},
 'plan': {'date': 1468450661000,
  'from': {'lat': 37.394535,
   'lon': -121.868908,
   'name': 'corner of Camino del Rey and North Capitol Avenue',
   'orig': '',
   'vertexType': 'NORMAL'},
  'itineraries': [{'duration': 6429,
    'elevationGained': 0.0,
    'elevationLost': 0.0,
    'endTime': 1468457090000,
    'legs': [{'agencyTimeZoneOffset': -25200000,
      'arrivalDelay': 0,
      'departureDelay': 0,
      'distance': 8373.988,
      'duration': 6429.0,
      'endTime': 1468457090000,
      'from': {'departure': 1468450661000,
       'lat': 37.394535,
       'lon': -121.868908,
       'name': 'corner of Camino del Rey and North Capitol Avenue',
       'orig': '',
       'vertexType': 'NORMAL'},
      'interlineWithPreviousLeg': False,
      'legGeometry': {'length': 198,
       'points': '{rfcF~oyfViApBYb@}@tAOTKPgAbBiBzC_@

In [ ]:
### Making a dictionary ###

data = {}
# OR
data = dict()

### Initially adding values ###

data = {'a':1,'b':2,'c':3}
# OR
data = dict(a=1, b=2, c=3)

### Inserting/Updating value ###

data['a']=1  # updates if 'a' exists, else adds 'a'
# OR
data.update({'a':1})
# OR
data.update(dict(a=1))
# OR
data.update(a=1)

### Merging 2 dictionaries ###

data.update(data2)  # Where data2 is also a dict.

### Deleting items in dictionary ###

del data[key] #Remove specific element in a dictionary
data.pop(key) #Removes the key & returns the value
data.clear() #Clear entire dictionary
